In [1]:
import time
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import urllib3
import requests

In [4]:
# Open the drive
driver = uc.Chrome()

In [5]:
query_url = 'https://ieeexplore.ieee.org/search/searchresult.jsp?action=search&matchBoolean=true&queryText=(((%22Abstract%22:%22Sentiment%20Analysis%22)%20AND%20(%22Abstract%22:%22Green%22%20OR%20%22Abstract%22:%22Nature%22%20OR%20%22Abstract%22:%22Environment%22%20OR%20%22Abstract%22:%22Chemical%22%20OR%20%22Abstract%22:%22Food%22%20OR%20%22Abstract%22:%22Plant%22%20OR%20%22Abstract%22:%22Organism%22%20OR%20%22Abstract%22:%22Climate%20Change%22%20OR%20%22Abstract%22:%22Sustainability%22%20OR%20%22Abstract%22:%22Sustainable%22%20OR%20%22Abstract%22:%22Carbon%22%20OR%20%22Abstract%22:%22Emission%22%20OR%20%22Abstract%22:%22Waste%22%20OR%20%22Abstract%22:%22Pollution%22%20OR%20%22Abstract%22:%22Global%20Warming%22)%20AND%20(%22Abstract%22:%22Corpora%22%20OR%20%22Abstract%22:%22Lexicon%22%20OR%20%22Abstract%22:%22Model%22%20OR%20%22Abstract%22:%22Algorithm%22%20OR%20%22Abstract%22:%22Classifier%22)))&highlight=true&returnType=SEARCH&matchPubs=true&refinements=ContentType:Conferences&refinements=ContentType:Journals&returnFacets=ALL&ranges=2012_2022_Year'

In [6]:
driver.get(query_url)
time.sleep(5)

In [ ]:
# Get all link for abstract query
list_paperlinks = []
try:
  driver.find_element_by_css_selector("[aria-label='dismiss cookie message'").click()
  time.sleep(5)
except:
  pass

# Get links from the first page
links = []
hrefs = driver.find_elements_by_xpath("//*[@class='List-results-items']//*[@href]")
for a in hrefs:
    link = a.get_attribute('href')
    if "document" in link:
        list_paperlinks.append(link)

# Get links from the next page    
while True:
    try:
        driver.find_element_by_class_name("next-btn").click()
        time.sleep(5)
        hrefs = driver.find_elements_by_xpath("//*[@class='List-results-items']//*[@href]")
        for a in hrefs:
            link = a.get_attribute('href')
            if "document" in link:
                list_paperlinks.append(link)
    except:
        break

In [9]:
driver.quit()

In [ ]:
# Wrap into a data frame and save to xlsx
list_source = []
list_querystring = []
list_querylinks = []
[list_source.append("IEEEXplore") for i in list_paperlinks]
[list_querystring.append("ALL") for i in list_paperlinks]
[list_querylinks.append(query_url) for i in list_paperlinks]
ieee_paperlinks = pd.DataFrame({'Source': list_source,'Query': list_querystring, 'Query Links': list_querylinks, 'Paper Links': list_paperlinks})
ieee_paperlinks.to_excel("./paper-links/PaperLinks_IEEEXplore_AllDuplicated.xlsx", index = False)
len(ieee_paperlinks)

In [ ]:
# Drop Duplicate
ieee_paperlinks_dd = ieee_paperlinks.drop_duplicates(['Paper Links'], keep='first')
ieee_paperlinks_dd = ieee_paperlinks_dd[ieee_paperlinks_dd["Paper Links"].str.contains('citations')==False]
ieee_paperlinks_dd.to_excel("./paper-links/PaperLinks_IEEEXplore.xlsx", index = False)
len(ieee_paperlinks_dd)